In [ ]:
%pip install pandas elasticsearch

In [17]:
from elasticsearch import Elasticsearch
import pandas as pd
from elasticsearch.helpers import bulk
import subprocess
import os


es = Elasticsearch("http://localhost:9200")

print(es.info())

{'name': 'ANTHIPPI', 'cluster_name': 'elasticsearch', 'cluster_uuid': 'spRS4hvpTO6ikxSrVs-Tiw', 'version': {'number': '9.1.5', 'build_flavor': 'default', 'build_type': 'zip', 'build_hash': '90ee222e7e0136dd8ddbb34015538f3a00c129b7', 'build_date': '2025-10-02T22:07:12.966975992Z', 'build_snapshot': False, 'lucene_version': '10.2.2', 'minimum_wire_compatibility_version': '8.19.0', 'minimum_index_compatibility_version': '8.0.0'}, 'tagline': 'You Know, for Search'}


In [4]:
file_path = '../data/documents.csv' 

try:
    df = pd.read_csv(file_path)
    print("Το αρχείο φορτώθηκε επιτυχώς!")
except FileNotFoundError:
    print(f"Πρόσοχή: Το αρχείο δεν βρέθηκε στη διαδρομή: {file_path}")


if 'Text' in df.columns:
    initial_rows = len(df)
    df = df.dropna(subset=['Text'])
    print(f"\nΑφαιρέθηκαν {initial_rows - len(df)} γραμμές με κενό κείμενο.")

    df['Text'] = df['Text'].astype(str)

if 'ID' in df.columns:
    df['ID'] = df['ID'].astype(str)


Το αρχείο φορτώθηκε επιτυχώς!

Αφαιρέθηκαν 0 γραμμές με κενό κείμενο.


In [5]:
index_name = "ir2025_index"

settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0,
        "analysis": {
            "analyzer": {
                "default": {
                    "type": "english" 
                }
            }
        }
    },
    "mappings": {
        "properties": {
            "doc_id": {"type": "keyword"}, 
            "text": {
                "type": "text",
                "similarity": "BM25"
            }
        }
    }
}

if es.indices.exists(index=index_name):
    es.indices.delete(index=index_name)

es.indices.create(index=index_name, body=settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'ir2025_index'})

In [6]:
index_name = "ir2025_index"

def generate_data(dataframe):
    for index, row in dataframe.iterrows():
        yield {
            "_index": index_name,
            "_source": {
                "doc_id": str(row['ID']), 
                "text": row['Text']
            }
        }

print("Ξεκινάει η εισαγωγή των κειμένων...")

success, failed = bulk(es, generate_data(df))

print(f"Ολοκληρώθηκε! Επιτυχίες: {success}, Αποτυχίες: {failed}")

es.indices.refresh(index=index_name)
print("Το ευρετήριο ανανεώθηκε και είναι έτοιμο για αναζήτηση.")

Ξεκινάει η εισαγωγή των κειμένων...
Ολοκληρώθηκε! Επιτυχίες: 18316, Αποτυχίες: []
Το ευρετήριο ανανεώθηκε και είναι έτοιμο για αναζήτηση.


In [ ]:

queries_path = '../data/queries.csv'
queries_df = pd.read_csv(queries_path)
queries_df.columns = queries_df.columns.str.strip() 

qid_col = 'ID' 
qtext_col = 'Text'


k_values = [20, 30, 50]

print(f"Ξεκινάει η διαδικασία για k = {k_values}...")


for k in k_values:
    

    output_file = f"../results/phase1_results_k{k}.txt"
    run_id = f"my_elastic_k{k}" 
    
    print(f"  -> Εκτέλεση αναζήτησης για k={k}...")
    
    with open(output_file, 'w') as f:
  
        for index, row in queries_df.iterrows():
            q_id = str(row[qid_col])
            q_text = row[qtext_col]
            
            if pd.isna(q_text):
                continue


            response = es.search(
                index="ir2025_index",
                body={
                    "query": {
                        "match": {
                            "text": q_text 
                        }
                    },
                    "size": k  
                }
            )

            hits = response['hits']['hits']
            
            for rank, hit in enumerate(hits):
                doc_id = hit['_source']['doc_id']
                score = hit['_score']
                
                line = f"{q_id}\tQ0\t{doc_id}\t{rank+1}\t{score:.4f}\t{run_id}\n"
                f.write(line)
                
    print(f"     Αποθηκεύτηκε στο: {output_file}")

print("\nΟλοκληρώθηκαν όλα τα πειράματα!")

Ξεκινάει η διαδικασία για k = [20, 30, 50]...
  -> Εκτέλεση αναζήτησης για k=20...
     Αποθηκεύτηκε στο: ../results/phase1_results_k20.txt
  -> Εκτέλεση αναζήτησης για k=30...
     Αποθηκεύτηκε στο: ../results/phase1_results_k30.txt
  -> Εκτέλεση αναζήτησης για k=50...
     Αποθηκεύτηκε στο: ../results/phase1_results_k50.txt

Ολοκληρώθηκαν όλα τα πειράματα!


In [18]:

path_to_trec_eval = "../../trec_eval/trec_eval.exe" 
qrels_file = "../data/qrels.txt"

k_values = [20, 30, 50]

print(f"Ξεκινάει η μαζική αξιολόγηση με trec_eval...")

for k in k_values:

    results_file = f"../results/phase1_results_k{k}.txt"
    
    if not os.path.exists(results_file):
        print(f"\n ΠΡΟΣΟΧΗ: Το αρχείο {results_file} δεν βρέθηκε. Το προσπερνάω.")
        continue

    print("\n" + "="*60)
    print(f"ΑΠΟΤΕΛΕΣΜΑΤΑ ΓΙΑ k = {k}")
    print(f"   Αρχείο: {results_file}")
    print("="*60)


    metrics_str = "P.5,10,15,20"
 
    command = [
        path_to_trec_eval,
        "-m", "map",           
        "-m", metrics_str,  
        qrels_file,
        results_file
    ]

    try:
        result = subprocess.run(
            command, 
            capture_output=True, 
            text=True, 
            check=True
        )
        
        print(result.stdout)
        
    except FileNotFoundError:
        print("ΣΦΑΛΜΑ: Δεν βρέθηκε το trec_eval.exe! Έλεγξε τη διαδρομή.")
        break 
    except subprocess.CalledProcessError as e:
        print(f"ΣΦΑΛΜΑ κατά την εκτέλεση για k={k}:")
        print(e.stderr)

Ξεκινάει η μαζική αξιολόγηση με trec_eval...

ΑΠΟΤΕΛΕΣΜΑΤΑ ΓΙΑ k = 20
   Αρχείο: ../results/phase1_results_k20.txt
map                   	all	0.6895
P_5                   	all	0.8800
P_10                  	all	0.7300
P_15                  	all	0.7000
P_20                  	all	0.6300


ΑΠΟΤΕΛΕΣΜΑΤΑ ΓΙΑ k = 30
   Αρχείο: ../results/phase1_results_k30.txt
map                   	all	0.7471
P_5                   	all	0.8800
P_10                  	all	0.7300
P_15                  	all	0.7000
P_20                  	all	0.6300


ΑΠΟΤΕΛΕΣΜΑΤΑ ΓΙΑ k = 50
   Αρχείο: ../results/phase1_results_k50.txt
map                   	all	0.7701
P_5                   	all	0.8800
P_10                  	all	0.7300
P_15                  	all	0.7000
P_20                  	all	0.6300

